In [ ]:
from google.colab import drive
drive.mount('insert_path_here')
import sys
sys.path
sys.path.append('insert_path_here')
sys.path.append('insert_path_here')
import numpy as np
import matplotlib.pyplot as plt
import Dans_Diffraction as dif
import re
import matplotlib.pyplot as plt
import pandas as pd
import io
import torch.nn as nn
import torch.optim as optim
import torch
dtype = torch.float32
device = torch.device("cpu")
import torch.nn.functional as F 


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import Data 
directory = 'insert_path_here'
shifted_phase = np.load(directory + 'phase_v2.npy')
shifted_amplitude = np.load(directory + 'amp_v2.npy')
shift_vector = np.load(directory + 'shift_v2.npy')

In [ ]:
shifted_phase = np.load(directory + 'phase_v2.npy')
shifted_amplitude = np.load(directory + 'amp_v2.npy')
shift_vector = np.load(directory + 'shift_v2.npy')

In [ ]:
# Create labels for the input data
correct_label_vector = np.zeros(shift_vector.shape[0])
for i in range(16):
  correct_label_vector[i*1000:(i+1)*1000] = i

0.0
2.0
12.0


In [ ]:
# Define training and validation sets
num_folds = 10
val_set_end = len(correct_label_vector)
val_set_begin = len(correct_label_vector) - int((len(correct_label_vector)/num_folds))
val_set_length = val_set_end - val_set_begin

shuffle_ind=np.random.permutation(shifted_amplitude.shape[0])
shuffled_amps=shifted_amplitude[shuffle_ind]
shuffled_circ=shifted_phase[shuffle_ind]
val_set_target = shuffled_circ[val_set_begin: val_set_end]
val_set_input = shuffled_amps[val_set_begin: val_set_end]
val_set_input = val_set_input.reshape(val_set_input.shape[0], 1, val_set_input.shape[1], val_set_input.shape[2])

train_set_target = shuffled_circ[0: val_set_begin]
train_set_input = shuffled_amps[0: val_set_begin]
train_set_input = train_set_input.reshape(train_set_input.shape[0], 1, train_set_input.shape[1], train_set_input.shape[2])


1600
1600
14400
14400


In [ ]:
# Convert data to lists
train_set_input = train_set_input.tolist()
val_set_input = val_set_input.tolist()
train_set_target = train_set_target.tolist()
val_set_target = val_set_target.tolist()

In [ ]:
# Convert data to maps
train_set_input = list(map(str, train_set_input))

In [ ]:
val_set_input = list(map(str, val_set_input))

In [ ]:
shuffled_amp_list = list(map(str, shuffled_amps))

In [ ]:
shuffled_circ_list = shuffled_circ.tolist()

In [ ]:
import torch
# Create Dataset class for input data
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, labels, shifted_amplitude):
        'Initialization'
        self.all_data = torch.from_numpy(shifted_amplitude)
        #self.labels = labels
        self.labels = labels
        #self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.all_data)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = self.all_data[index]

        # Load data and get label
        y = self.labels[index]

        return X, y
